In [ ]:
# !pip install kaggle

In [ ]:
# from google.colab import files
# files.upload()

In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !kaggle datasets download -d kazanova/sentiment140

In [ ]:
# !unzip '/content/sentiment140.zip' -d '/content/'

Required **Imports**

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from pandas import Series, DataFrame
plt.style.use('ggplot')
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.cm as cm
from matplotlib import rcParams
from collections import Counter
from sklearn.model_selection import train_test_split
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from sklearn.utils import shuffle
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from math import *
from sklearn.metrics import *
from sklearn.metrics import confusion_matrix
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv("/content/sample_data/training.1600000.processed.noemoticon.csv", encoding = 'ISO-8859-1')
data_2=data
data_3=data
data.head()

In [ ]:
colmns = data.columns

In [ ]:
data['text'] = data[data.columns[-1]]
data['label'] = data[data.columns[0]]

In [ ]:
data = data.drop(columns=colmns)

In [ ]:
data.head()

In [ ]:
def remove_spaces(text):
  return re.sub(' +',' ',text)

data["text"] = data["text"].apply(lambda text: remove_spaces(text))

data.head()

In [ ]:
# 2. Removing URLs
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

data["text"] = data["text"].apply(lambda text: remove_urls(text))

data.head()

In [ ]:
# 3. Removing Emojies

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

data["text"] = data["text"].apply(lambda text: remove_emoji(text))

data.head()

In [ ]:


PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

data["text"] = data["text"].apply(lambda text: remove_punctuation(text))

data.head()

In [ ]:
# 5. Removing multiple spaces


data["text"] = data["text"].apply(lambda text: " ".join(text.split()))



In [ ]:

# 6. Removing non english words
nltk.download('words')
wds = set(nltk.corpus.words.words())

data['text'] = data['text'].apply(lambda text: " ".join(w for w in nltk.wordpunct_tokenize(text) \
         if w.lower() in wds))


In [ ]:
# 7. Removing numbers
def rem_num(text):
  return ''.join(i for i in text if not i.isdigit())

data['text'] = data["text"].apply(lambda text: rem_num(text))


In [ ]:
#Fake targets

c=data.groupby('label')
k=c.get_group(0)
k.head()

In [ ]:
# Real targets

p=c.get_group(4)
p.head()

In [ ]:
all_words = ' '.join([text for text in k.text])
wordcloud = WordCloud(width= 800, height= 800, max_font_size = 115, collocations = False).generate(all_words)
plt.figure(figsize=(10,7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
all_words = ' '.join([text for text in p.text])
wordcloud = WordCloud(width= 800, height= 800, max_font_size = 150, collocations = False).generate(all_words)
plt.figure(figsize=(10,7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
data.head()

In [ ]:
tweets =  shuffle(data).reset_index(drop=True)

In [ ]:
tokenized_tweet=tweets['text'].apply(lambda x: x.split())
tokenized_tweet.head(5)

In [ ]:

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts = cv.fit_transform(tweets['text'].values.astype('U'))

In [ ]:
X=text_counts
y=tweets['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=19)

using naive bayes

In [ ]:

cnb = ComplementNB()
cnb.fit(X_train, y_train)
cross_cnb = cross_val_score(cnb, X, y,n_jobs = -1)
print("Cross Validation score = ",cross_cnb)                
print ("Train accuracy ={:.2f}%".format(cnb.score(X_train,y_train)*100))
print ("Test accuracy ={:.2f}%".format(cnb.score(X_test,y_test)*100))
train_acc_cnb=cnb.score(X_train,y_train)
test_acc_cnb=cnb.score(X_test,y_test)

In [ ]:
print("plotting the best parameters")
data_cnb = [train_acc_cnb,test_acc_cnb]
labels = ['Train Accuracy','Test Accuracy']
plt.xticks(range(len(data_cnb)), labels)
plt.ylabel('Accuracy')
plt.title('Accuracy plot with best parameters')
plt.bar(range(len(data_cnb)), data_cnb,color=['blue','darkorange']) 
Train_acc = mpatches.Patch(color='blue', label='Train_acc')
Test_acc = mpatches.Patch(color='darkorange', label='Test_acc')
plt.legend(handles=[Train_acc, Test_acc],loc='best')
plt.gcf().set_size_inches(8, 8)
plt.show()

In [ ]:
y_pred_cnb =cnb.predict(X_test)
print("Confusion matrix : ")
print(confusion_matrix(y_test,y_pred_cnb))

In [ ]:
print(classification_report(y_test, y_pred_cnb))

roc_score_cnb=roc_auc_score(y_test, y_pred_cnb)
print("Area Under the Curve = ",roc_score_cnb)

In [ ]:
f1_scre=f1_score(y_test, y_pred_cnb, average="macro")
final_f1S=f1_scre*100
print("F1 score =",final_f1S)
print()
print("**************************************")
print()
f1_cnb=f1_score(y_test, y_pred_cnb, average="macro")
precisio_scre=precision_score(y_test, y_pred_cnb, average="macro")
finalPreScore=100*precisio_scre
print("Precision score :  ",finalPreScore)
print()
print("******************************************")
precision_cnb=precision_score(y_test, y_pred_cnb, average="macro")
print("precision_cnb : ")
print(precision_cnb)
print("******************************************")
print()
recScre=recall_score(y_test, y_pred_cnb, average="macro")
final_recal=100*recScre
print("Recall score :",final_recal)  
recall_cnb=recall_score(y_test, y_pred_cnb, average="macro")

Now WE will use tensorflow_based_model NN

In [ ]:
data_2 = pd.read_csv("/content/sample_data/training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1")
data_2.columns = ["label", "time", "date", "query", "username", "text"]


In [ ]:
data_2=data_2[['text','label']]

In [ ]:
data_2['label'][data_2['label']==4]=1

In [ ]:
data_pos = data_2[data_2['label'] == 1]
data_neg = data_2[data_2['label'] == 0]

In [ ]:
data_pos = data_pos.iloc[:int(24500)]
data_neg = data_neg.iloc[:int(24500)]

In [ ]:
data_2 = pd.concat([data_pos, data_neg])

In [ ]:
data_2['text']=data_2['text'].str.lower()

In [ ]:
def cleaning_email(data_2):
    return re.sub('@[^\s]+', ' ', data_2)

In [ ]:
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [ ]:
def cleaning_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [ ]:
def cleaning_numbers(data_2):
    return re.sub('[0-9]+', '', data_2)

In [ ]:
def cleaning_URLs(data_2):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',data_2)

In [ ]:
st = nltk.PorterStemmer()
def stemming_on_text(data_2):
    text = [st.stem(word) for word in data_2]
    return data_2

In [ ]:
def tensorflow_based_model(): 
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(2000,50,input_length=max_len)(inputs)
    layer = LSTM(63)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
data_2['text']= data_2['text'].apply(lambda x: cleaning_punctuations(x))
data_2['text'].tail()

In [ ]:
data_2['text'] = data_2['text'].apply(lambda x: cleaning_repeating_char(x))
data_2['text'].tail()

In [ ]:
data_2['text']= data_2['text'].apply(lambda x: cleaning_email(x))
data_2['text'].tail()

In [ ]:
data_2['text'] = data_2['text'].apply(lambda x: cleaning_URLs(x))
data_2['text'].tail()

In [ ]:
data_2['text'] = data_2['text'].apply(lambda x: cleaning_numbers(x))
data_2['text'].tail()

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
data_2['text'] = data_2['text'].apply(tokenizer.tokenize)

In [ ]:
data_2['text'].head()

In [ ]:
data_2['text']= data_2['text'].apply(lambda x: stemming_on_text(x))

In [ ]:
data['text'].head()

In [ ]:
data_2['text'].head()

In [ ]:
X=data_2.text
y=data_2.label

In [ ]:
max_len = 470
tok = Tokenizer(num_words=1999)
print(tok)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
sequences_matrix.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(sequences_matrix, y, test_size=0.3, random_state=2)

In [ ]:
model = tensorflow_based_model()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])  
print(model)

In [ ]:
history=model.fit(X_train,Y_train,batch_size=84,epochs=3, validation_split=0.1)
print(history)

In [ ]:
accr1 = model.evaluate(X_test,Y_test)
print(accr1)

In [ ]:
print("-----------the accuracy of the model on test data is given below-----------")
print("Test set")
print()
print("Accuracy: ",accr1[1])
print()
print()

In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5) 

In [ ]:
print('\n')
print("confusion matrix")
print('\n')
CR=confusion_matrix(Y_test, y_pred)
print(CR)
print('\n')


In [ ]:
fpr, tpr, thresholds = roc_curve(Y_test, y_pred)
area = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=1, label='ROC curve (area = %0.3f)' % area)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC')
plt.legend(loc="upper right")
plt.show()

In [ ]:
data_3 = pd.read_csv("/content/sample_data/training.1600000.processed.noemoticon.csv", encoding = 'ISO-8859-1')


In [ ]:
data_3.head()

In [ ]:
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "TweetText"]
data_3.columns = DATASET_COLUMNS
data_3.head()

In [ ]:
data_3.drop(['ids','date','flag','user'],axis = 1,inplace = True)


In [ ]:
positif_data = data_3[data_3.target==4].iloc[:24500,:]

negative_data = data_3[data_3.target==0].iloc[:900,:]
print()

print("**********positif_data**********")
print(positif_data.shape)

print()

print("*********negative_data***********")
print(negative_data.shape)

In [ ]:
print("positif_data and negative_data")
tweet_df = pd.concat([positif_data,negative_data],axis = 0)
tweet_df.tail()

In [ ]:
100*tweet_df.isna().sum()/len(tweet_df)

In [ ]:
def cleanup_tweets(tweet_df):
    tweet_df['clean_tweet'] = tweet_df['TweetText'].str.replace("@", "")
    tweet_df['clean_tweet'] = tweet_df['clean_tweet'].str.replace("[^a-zA-Z]", " ")
    tweet_df['clean_tweet'] = tweet_df['clean_tweet'].str.replace(r"http\S+", "")      
    tweet_df['clean_tweet'] = tweet_df['clean_tweet'].apply(lambda x: x.split())
    stemmer = PorterStemmer()
    tweet_df['clean_tweet'] = tweet_df['clean_tweet'].apply(lambda x: [stemmer.stem(i) for i in x])
    tweet_df['clean_tweet'] = tweet_df['clean_tweet'].apply(lambda x: ' '.join([w for w in x]))
    tweet_df['clean_tweet'] = tweet_df['clean_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [ ]:
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud
from wordcloud import STOPWORDS

In [ ]:
cleanup_tweets(tweet_df)

In [ ]:
ls=0
def createWrdCloudForSentiment(target):
    temp_df = pd.DataFrame() 
    if target == -1:
        temp_df = tweet_df
        ls=8
    else:
        temp_df = tweet_df[tweet_df.target==target]
    words = " ".join(temp_df.clean_tweet)
    wrdcld = WordCloud(stopwords=STOPWORDS,
                      background_color='black',
                      width=1500,
                      height=1000).generate(words)
    plt.figure(figsize=(10,10))
    plt.imshow(wrdcld)
    plt.axis('off')
    plt.show

In [ ]:
createWrdCloudForSentiment(-1)

In [ ]:
print("***************tweets for positive sentiment*******************")
createWrdCloudForSentiment(4) 

In [ ]:
print("*************tweet for depressive sentiments*************")
createWrdCloudForSentiment(0) 

In [ ]:
def createPieChartFor(t_df):
    a=len(t_df)
    b=t_df.value_counts()/a
    Lst = 100*b
    
    labels = t_df.value_counts().index.values
    sizes =  Lst 
    
    fig1, ax1 = plt.subplots()
    ax1.pie(sizes, labels=labels, autopct='%1.2f%%', startangle=100)
    ax1.axis('equal')
    plt.show()

In [ ]:
train_df, test_df = train_test_split(tweet_df, test_size=0.3, random_state=33)

In [ ]:
test_tweets =[]
train_tweets =[]
i=0
for tweet in train_df.clean_tweet:
    train_tweets.append(tweet)
    i=+1
    
for tweet in test_df.clean_tweet:
    test_tweets.append(tweet)

In [ ]:
train_tweets[:9]

In [ ]:
vectorizer = TfidfVectorizer()
train_tfidf_model = vectorizer.fit_transform(train_tweets)
test_tfidf_model = vectorizer.transform(test_tweets)

In [ ]:
train_tfidf = pd.DataFrame(train_tfidf_model.toarray(), columns=vectorizer.get_feature_names())
train_tfidf.head()

In [ ]:
i=0
j=0
cls = [MultinomialNB(), 
       RandomForestClassifier(n_estimators=999, random_state=25),
       svm.SVC()]
accuracy = []
cls_name = []
lbl_actual = test_df.target
for cl in cls:
    j+=1
    model = cl.fit(train_tfidf_model,train_df.target)
    lbl_pred = model.predict(test_tfidf_model)
    a = (100*accuracy_score(lbl_pred, lbl_actual))
    a = round(a,2)
    accuracy.append(a)
    cls_name.append(cl.__class__.__name__)
    print("********************")
    print(cls_name[i])
    print("Accuracy Score : ")
    print(a)
    print("********************")
    print( classification_report(lbl_pred, lbl_actual))
    i =i+1

In [ ]:
print("Accuracy for Naive Bayes, Random Forest Classifier and SVM")
accuracy

In [ ]:
plt.figure(figsize=(10,9))
plt.bar(cls_name, accuracy,color='blue')
plt.xticks(rotation=70)
for index,data in enumerate(accuracy):
    plt.text(x=index , y =data+1 , s=f"{data}%" , fontdict=dict(fontsize=11),color='red')
plt.tight_layout()
plt.show()

In [ ]:
i = 0
cls = [LogisticRegression(),
       DecisionTreeClassifier(),
       KNeighborsClassifier(n_neighbors = 5),
       XGBClassifier(max_depth=5, n_estimators=999, nthread= 3)]
accuracy = []
cls_name = []
lbl_actual = test_df.target


for cl in cls:
    model = cl.fit(train_tfidf_model,train_df.target)
    lbl_pred = model.predict(test_tfidf_model)
    a = (100*accuracy_score(lbl_pred, lbl_actual))
    a = round(a,2)
    accuracy.append(a)
    cls_name.append(cl.__class__.__name__)
    print("********************")
    print(cls_name[i])
    print("Accuracy Score : ")
    print(a)
    print("********************")
    print( classification_report(lbl_pred, lbl_actual))
    i =i+1



In [ ]:
print("Accuracy for Logistic Regression, Decision Tree, K-Neighbors ,XG Boast")

In [ ]:
plt.figure(figsize=(10,9))
plt.bar(cls_name, accuracy,color='blue')
plt.xticks(rotation=70)
for index,data in enumerate(accuracy):
    plt.text(x=index , y =data+1 , s=f"{data}%" , fontdict=dict(fontsize=11),color='red')
plt.tight_layout()
plt.show()